In [64]:
import sys
from pathlib import Path
import pandas as pd 
import numpy as np
from scipy.stats import pearsonr


sys.path.append(str(Path("../../..").resolve()))

from src.constants import processed_data_dir, raw_data_dir, raw_data_name

In [36]:
df_x = pd.read_excel(processed_data_dir / "imputed_na_value_data.xlsx")
df_x.drop(df_x.index[len(df_x)-1], inplace=True)
df_y = pd.read_excel(raw_data_dir / raw_data_name, sheet_name=3)[["Date", "value_1d_fwd_rel_ret"]]

df_all = pd.concat([df_y, df_x], axis=1)
df_all.head()

,Date,value_1d_fwd_rel_ret,VIX,V2X,MSCI EM Vol,MOVE,Oil Vol,Gold Vol,HY Vol,JPM Global FX Vol,...,EURUSD ATM Vol 1Y-3M,G10 Carry Trade Index,JPY/USD Carry,GBP/USD Carry,EUR/USD Carry,CAD/USD Carry,AUD/USD Carry,NZD/USD Carry,CHF/USD Carry,Copper 1 year carry
0,2000-05-30,-0.004145,23.62,25.63,0.0,105.47,0.0,0.0,0.0,12.21,...,13.400,110.765,98.7917,90.2306,76.1611,101.1919,93.7378,86.1437,77.0754,0.0
1,2000-05-31,0.004223,23.65,25.26,0.0,103.10,0.0,0.0,0.0,12.11,...,13.275,110.382,97.7475,90.5178,76.8011,101.7900,93.2954,85.8620,77.4453,0.0
2,2000-06-01,-0.003080,22.36,24.14,0.0,101.26,0.0,0.0,0.0,11.95,...,13.175,110.611,96.9475,90.0934,76.2372,102.1823,93.2278,85.7492,77.1084,0.0
3,2000-06-02,0.004965,21.48,22.76,0.0,98.23,0.0,0.0,0.0,11.96,...,13.400,110.789,97.3623,91.0199,77.4666,103.1199,94.4805,87.2503,78.2057,0.0
4,2000-06-05,-0.005362,22.71,23.45,0.0,96.08,0.0,0.0,0.0,11.86,...,13.175,110.838,97.9722,91.6087,77.5905,102.9840,94.8368,87.8504,78.2794,0.0


## Explore ACF

In [44]:
auto_cor = [{"lag": lag, "corr": df_all["value_1d_fwd_rel_ret"].autocorr(lag)} for lag in range(1, 90)]
pd.DataFrame(auto_cor).sort_values("corr", ascending=False).head()

,lag,corr
0,1,0.064867
26,27,0.062619
38,39,0.053404
84,85,0.039863
58,59,0.039070


In [73]:
def cross_cor(df = df_all, target = "value_1d_fwd_rel_ret", lag = 1):
    corr_dict= {}
    for col in df.columns.tolist():
        if col not in ["Date", "value_1d_fwd_rel_ret"]:
            corr_dict[col] = pearsonr(df[target][lag:], df[col].shift(lag)[lag:])[0]
    
    return corr_dict

In [83]:
cross_corrs = pd.DataFrame([cross_cor(df_all, "value_1d_fwd_rel_ret", lag )for lag in range(1, 90)])

cross_corrs.max().sort_values()

CDX HY 5Yr                      -0.045202
MSCI USA Enhanced Value Index   -0.043655
iTraxx Crossover 5Yr            -0.043122
MSCI DM                         -0.042741
MSCI EM                         -0.041854
                                   ...   
EURUSD ATM Vol 1Y-3M             0.051297
JPM Global FX Vol                0.054313
JPM G7 FX Vol                    0.057015
USDJPY 1M Vol                    0.058011
EURUSD 1M Vol                    0.058322
Length: 110, dtype: float64

## Feature selection